In [1]:
!pip install -q -U google-generativeai

In [2]:
import pandas as pd
import numpy as np
import re

# Import the Python SDK
import google.generativeai as genai
# Used to securely store your API key
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [3]:
def clean_str(string):
  string = re.sub(r"\'s", "", string)
  string = re.sub(r"\'ve", "", string)
  string = re.sub(r"n\'t", "", string)
  string = re.sub(r"\'re", "", string)
  string = re.sub(r"\'d", "", string)
  string = re.sub(r"\'ll", "", string)
  string = re.sub(r",", "", string)
  string = re.sub(r"!", " ! ", string)
  string = re.sub(r"\(", "", string)
  string = re.sub(r"\)", "", string)
  string = re.sub(r"\?", "", string)
  string = re.sub(r"'", "", string)
  string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
  string = re.sub(r"[0-9]\w+|[0-9]","", string)
  string = re.sub(r"\s{2,}", " ", string)
  return string.strip()

model = genai.GenerativeModel('gemini-2.0-flash-001')

In [4]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Veridion/Data/ml_insurance_challenge.csv')
df_labels = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Veridion/Data/insurance_taxonomy - insurance_taxonomy.csv')

data['description'] = data['description'].fillna('None')

labels = ''
for index, row in df_labels.iterrows():
  labels += row['label'] + ','

In [5]:
predictions = []

for index, row in data.iterrows():
  prediction = []
  prompt = "Given the list of labels: " + str(labels) + \
            'and a company with the descripion:' + str(row['description']) + \
            ', tags:' +  str(row['business_tags']) + \
            ', sector:' +  str(row['sector']) + \
            ', category:' +  str(row['category']) + \
            ', niche:' +  str(row['niche']) + '/n' +\
            'Can you tell me what labels fit best? Tell me just the labls in a list separated by , like (a, b, c)'
  print("Processing: ", index)
  response = model.generate_content(prompt)

  for element in re.split(',', response.text):
    prediction.append(clean_str(element))
  predictions.append(prediction)


Streaming output truncated to the last 5000 lines.
Processing:  4494
Processing:  4495
Processing:  4496
Processing:  4497
Processing:  4498
Processing:  4499
Processing:  4500
Processing:  4501
Processing:  4502
Processing:  4503
Processing:  4504
Processing:  4505
Processing:  4506
Processing:  4507
Processing:  4508
Processing:  4509
Processing:  4510
Processing:  4511
Processing:  4512
Processing:  4513
Processing:  4514
Processing:  4515
Processing:  4516
Processing:  4517
Processing:  4518
Processing:  4519
Processing:  4520
Processing:  4521
Processing:  4522
Processing:  4523
Processing:  4524
Processing:  4525
Processing:  4526
Processing:  4527
Processing:  4528
Processing:  4529
Processing:  4530
Processing:  4531
Processing:  4532
Processing:  4533
Processing:  4534
Processing:  4535
Processing:  4536
Processing:  4537
Processing:  4538
Processing:  4539
Processing:  4540
Processing:  4541
Processing:  4542
Processing:  4543
Processing:  4544
Processing:  4545
Processing:  

In [56]:
Pred = []
for prediction in predictions:
  string = ''
  for element in prediction:
    string += element + ','

  string = re.sub(',,', ',', string)
  if string[-1] == ',':
    string = string[:-1]
  string = re.sub(',', ', ', string)
  Pred.append(string)

df_prediction = pd.DataFrame(Pred, columns=['Prediction'])
df_prediction.to_csv('/content/drive/MyDrive/Colab Notebooks/Veridion/Data/Prediction.csv')

df_prediction.head()

,Prediction
0,"Pipeline Construction Services, Excavation Ser..."
1,"Food Processing Services, Seed Processing Serv..."
2,"Dairy Production Services, Bakery Production S..."
3,"Restoration Services, Painting Services"
4,"Tourism Services, Recreational Activities, Cul..."
